In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.models import resnet50
from torch import nn, optim
from torch.nn import CrossEntropyLoss
from PIL import Image
import os
from tqdm import tqdm

# Определение датасета
class ImageFolderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.class_to_idx = {}
        idx = 0
        for class_name in sorted(os.listdir(root_dir)):
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                self.class_to_idx[class_name] = idx
                idx += 1
                for img_file in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_file)
                    self.samples.append((img_path, self.class_to_idx[class_name]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


In [ ]:
# Трансформация изображений
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Создание датасета
dataset = ImageFolderDataset(r"C:\Users\user\Desktop\госкатлог\train_train_2", transform=transform)

# Разделение датасета
train_size = int(0.9 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# Загрузчики данных
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)



In [ ]:
# Определение модели
class SimpleResNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.base_model = resnet50(pretrained=True)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.base_model(x)

num_classes = len(dataset.class_to_idx)
model = SimpleResNet(num_classes)

# Устройство и оптимизация
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)



In [ ]:
# Цикл обучения и валидации
min_valid_loss = float('inf')
for epoch in range(5):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} [Training]"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Training Loss: {avg_train_loss}")

    model.eval()
    with torch.no_grad():
        valid_loss = 0
        for images, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1} [Validation]"):
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            loss = criterion(logits, labels)
            valid_loss += loss.item()

    avg_valid_loss = valid_loss / len(valid_loader)
    print(f"Validation Loss: {avg_valid_loss}")

    # Сохранение модели при улучшении валидационной ошибки
    if avg_valid_loss < min_valid_loss:
        print("Validation loss decreased, saving model...")
        torch.save(model.state_dict(), 'best_model-8b-256pic.pth')
        min_valid_loss = avg_valid_loss


In [ ]:
model = torch.load("best_model.pth")


In [ ]:
from torchvision import transforms
from PIL import Image
import torch

def predict_image(image_path, model, transform, device):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)  # Применение тех же трансформаций
    image = image.unsqueeze(0).to(device)  # Добавить batch dimension и перенести на устройство

    model.eval()  # Перевод модели в режим оценки
    with torch.no_grad():  # Отключить градиенты для инференса
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()

# Загрузка модели (если модель сохранена)
# model.load_state_dict(torch.load('path_to_saved_model.pth'))
model = model.to(device)  # Убедитесь, что модель находится на нужном устройстве

# Предполагается, что устройство и трансформация уже определены
predicted_class_index = predict_image("C:\\Users\\user\\Desktop\\госкатлог\\transformer\\3582715.jpg", model, transform, device)
print(f'Predicted class index: {predicted_class_index}')
print(f'Predicted class name: {list(dataset.class_to_idx.keys())[predicted_class_index]}')

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision.models import resnet50
from torch import nn
from torchvision import transforms
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Определение модели
class SimpleResNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.base_model = resnet50(pretrained=True)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.base_model(x)

# Загрузка весов модели
model = SimpleResNet(num_classes)
model.load_state_dict(torch.load('best_model.pth'))
model = model.to(device)
model.eval()

# Функция для получения предсказаний и истинных меток
def get_predictions(model, dataloader, device):
    predictions = []
    true_labels = []
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.view(-1).tolist())
            true_labels.extend(labels.view(-1).tolist())
    return true_labels, predictions

# Получение предсказаний и истинных меток
true_labels, predictions = get_predictions(model, valid_loader, device)

# Расчет матрицы ошибок
conf_matrix = confusion_matrix(true_labels, predictions)

# Визуализация матрицы ошибок
def plot_confusion_matrix(conf_matrix, class_names):
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(conf_matrix, annot=True, fmt="d", ax=ax, cmap='Blues', linewidths=.5)
    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')
    ax.set_title('Confusion Matrix')
    ax.xaxis.set_ticklabels(class_names)
    ax.yaxis.set_ticklabels(class_names)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.show()

# Получение названий классов
class_names = list(dataset.class_to_idx.keys())
plot_confusion_matrix(conf_matrix, class_names)


In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, f1_score
from torchvision.models import resnet50
from torch import nn

# Подразумевается, что модель уже загружена и находится в переменной `model`
# и что `valid_loader` и `device` также уже определены.

# Функция для получения предсказаний и истинных меток
def get_predictions(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    return true_labels, predictions

# Получение предсказаний и истинных меток
true_labels, predictions = get_predictions(model, valid_loader, device)

# Расчет F1-скора
f1 = f1_score(true_labels, predictions, average='weighted')  # Используется взвешенное среднее, если классы несбалансированы

# Вывод F1-скора
print(f'F1-Score (Weighted): {f1}')
